In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import imutils

import time
import math

%matplotlib inline
from matplotlib import pyplot as plt

# application config

In [2]:
EnableColorStream = True
ColorStreamResWidth = 1280
ColorStreamResHeight = 720
ColorStreamFPS = 30

EnableInfraredStream = False
InfraredStreamResWidth = 1280
InfraredStreamResHeight = 720
InfraredStreamFPS = 30

EnableDepthStream = True
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

maxDistance = 7000 # 1 meter

font = cv2.FONT_HERSHEY_TRIPLEX
textString = ''
colorFPS = (255,255,0)
colorDist = (0, 255,255)
colorRect = (0, 255,255)

colorLine = (0, 255, 0)
sizeLine = 3
colorLineP = (0, 255, 255)
sizeLineP = 1

# RealSense camera config

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream is True:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableInfraredStream is True:
    config.enable_stream(
        rs.stream.infrared, InfraredStreamResWidth, InfraredStreamResHeight, rs.format.bgr8, InfraredStreamFPS)
    
if EnableDepthStream is True:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
cfg = pipeline.start(config)

# Alignment
if EnableColorStream is True:
    align_to = rs.stream.color
elif EnableInfraredStream is True:
    align_to = rs.stream.infrared

align = rs.align(align_to)
    
# Advanced settings
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)

# Get depth scale
scale = depth_sensor.get_depth_scale()
print("depth scale:" + str(scale))
# PointCloud settings
#pc = rs.pointcloud()

depth scale:0.0010000000474974513


# color map function

In [4]:
def procColorMap(input_image):
    inp = input_image.copy()
    minNum, maxNum, minLoc, maxLoc = cv2.minMaxLoc(inp)

    if maxNum > maxDistance:
        maxNum = maxDistance
    
    scaleAlpha = 255 / maxNum
    inp = cv2.convertScaleAbs(inp, None, scaleAlpha, 0)
    depth_colormap = cv2.applyColorMap(inp, cv2.COLORMAP_JET)
    
    return depth_colormap

# screen capture function

In [5]:
# scrren capture global parameters
flagCapture = False

In [6]:
def procScreenCapture():  
    global flagCapture
    if flagCapture is True:
        flagCapture = False
        if EnableColorStream is True:
            fileName = "../imageCapture/color_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, color_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableInfraredStream is True:
            fileName = "../imageCapture/infrared_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, infrared_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableDepthStream is True:
            fileName = "../imageCapture/depth_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, depth_colormap, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


# measure distace and sectional drawing function

In [7]:
# measure distance global parameters
flagMeasure = False
flagDispMeasure= False
flagClickMeasure = False

measureCordLTX = -1
measureCordLTY = -1
measureCordRDX = -1
measureCordRDY = -1

measureImg = np.zeros((512,512,3), np.uint8)
measureImg2 = np.zeros((512,512,3), np.uint8)

In [8]:
# mouse click function
def procMouseMeasure(event,x,y,flags,param):
    global measureCordLTX, measureCordLTY, measureCordRDX, measureCordRDY
    global flagClickMeasure
    global measureImg, measureImg2
    
    if event is cv2.EVENT_LBUTTONDOWN:
        measureCordLTX = x
        measureCordLTY = y
        flagClickMeasure = True
        measureImg2 = measureImg.copy()
        
    elif event is cv2.EVENT_MOUSEMOVE:
        if flagClickMeasure is True:
            measureCordRDX = x
            measureCordRDY = y

    elif event is cv2.EVENT_LBUTTONUP:
        measureCordRDX = x
        measureCordRDY = y
        flagClickMeasure = False

In [9]:
# sectional drawing function
def drawSect(secLTX, secLTY, secRDX, secRDY):
    global depth_image
    output = []
    
    xdiff = secRDX - secLTX
    ydiff = secLTY - secRDY
    
    if xdiff < ydiff:
        for i in range(0, ydiff):
            posX = math.floor(secLTX + i * xdiff / ydiff)
            posY = math.floor(secRDY + i)
            data = depth_image[posY, posX]
            output.append(data)
            #print('x:' + str(posX) + ' y:' + str(posY) + ' z:' + str(data))
    else:
        for i in range(0, xdiff):
            posX = math.floor(secLTX + i)
            posY = math.floor(secRDY + i * ydiff / xdiff)
            data = depth_image[posY, posX]
            output.append(data)
            #print('x:' + str(posX) + ' y:' + str(posY) + ' z:' + str(data))
    
    cv2.namedWindow('Sectional drawing')
    #print(len(output))
    
    minOut = None
    maxOut = None
    for value in output:
        if not minOut:
            minOut = value
        elif value < minOut:
            minOut = value
        
        if not maxOut:
            maxOut = value
        elif value > maxOut:
            maxOut = value
      
    #print('min:' + str(minOut) + ' max:' + str(maxOut))
    
    parm = 620 / maxOut
    imgSec = np.zeros((720, len(output), 3), np.uint8)
    
    for i in range(0, len(output)):
        cv2.line(imgSec, (i, maxOut), (i,  int((maxOut - output[i])*parm)+100 ), colorLineP, 1)
    
    imgSec = cv2.resize(imgSec, (1280, 720), interpolation = cv2.INTER_CUBIC)
    cv2.imshow('Sectional drawing', imgSec)

In [10]:
# mouse flagMeasure function
def procMeasure():
    global flagMeasure, flagDispMeasure, flagClickMeasure
    global measureCordLTX, measureCordLTY, measureCordRDX, measureCordRDY
    global measureImg, measureImg2
    
    if flagMeasure is True:
        if EnableColorStream is True:
            measureImg = color_image.copy()
        elif EnableInfraredStream is True:
            measureImg = infrared_image.copy()
        cv2.namedWindow('Measure Distance')
        cv2.setMouseCallback('Measure Distance', procMouseMeasure)
        cv2.imshow('Measure Distance', measureImg)
        flagDispMeasure= True
        flagMeasure = False
    
    if flagDispMeasure is True:
        if flagClickMeasure is True:
            measureImg2 = measureImg.copy()
            if measureCordLTX != -1 and measureCordRDX != -1:
                cv2.line(measureImg2, (measureCordLTX,measureCordLTY), (measureCordRDX, measureCordRDY), colorLineP, sizeLineP)
            cv2.imshow('Measure Distance', measureImg2)
        else:
            if measureCordLTX != -1 and measureCordRDX != -1:
                cv2.line(measureImg, (measureCordLTX,measureCordLTY), (measureCordRDX, measureCordRDY), colorLine, sizeLine)
                textString = 'distance:'
                textX = int((measureCordLTX+measureCordRDX)/2)
                textY = int((measureCordLTY+measureCordRDY)/2)
                #cv2.rectangle(img, (textX-155, textY-35), (textX+155, textY+35), colorRect, -1)
                #cv2.rectangle(img, (textX-150, textY-30), (textX+150, textY+30), (255,255,255), -1)
                cv2.putText(measureImg, textString, ( textX-100, textY), font, 1, colorDist, 1, cv2.LINE_AA)
                drawSect(measureCordLTX, measureCordLTY, measureCordRDX, measureCordRDY)
                measureCordLTX = -1
                measureCordLTY = -1
                measureCordRDX = -1
                measureCordRDY = -1


            cv2.imshow('Measure Distance', measureImg)

# grabcut function

In [11]:
# grabcut global parameters
flagGrab = False
flagDispGrab = False
flagClickGrab = False

flagPaintYellow = False
flagPaintRed = False
flagRect = False

grabCordLTX = -1
grabCordLTY = -1
grabCordRDX = -1
grabCordRDY = -1

grabImg = np.zeros((512,512,3), np.uint8)
grabImg2 = np.zeros((512,512,3), np.uint8)

grabPaintX = -1
grabPaintY = -1

# mask = np.zeros(img.shape[:2], np.uint8)
mask = np.zeros((512,512,3), np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (-1, -1, -1, -1)
thickness = 3 

In [12]:
# mouse click function
def procMouseGrab(event,x,y,flags,param):
    global grabCordLTX, grabCordLTY, grabCordRDX, grabCordRDY
    global grabPaintX, grabPaintY
    global flagClickGrab, flagPaintYellow, flagPaintRed, flagRect
    global grabImg, grabImg2, grabImg3
    
    if event is cv2.EVENT_LBUTTONDOWN:
        if flagRect is True:
            if flags & cv2.EVENT_FLAG_CTRLKEY:
                grabPaintX = x
                grabPaintY = y
                flagPaintYellow = True
            elif flags & cv2.EVENT_FLAG_SHIFTKEY:
                grabPaintX = x
                grabPaintY = y
                flagPaintRed = True
        else:      
            if grabCordLTY != -1 and grabCordRDY != -1:
                grabImg = infrared_image.copy()
                grabImg3 = infrared_image.copy()

            grabCordLTX = x
            grabCordLTY = y
            flagClickGrab = True
            grabImg2 = grabImg.copy()
        
    elif event is cv2.EVENT_MOUSEMOVE:
        if flagClickGrab is True:
            grabCordRDX = x
            grabCordRDY = y
        elif flagPaintYellow is True or flagPaintRed is True:
            grabPaintX = x
            grabPaintY = y

    elif event is cv2.EVENT_LBUTTONUP:
        if flagPaintYellow is True:
            grabPaintX = x
            grabPaintY = y
            flagPaintYellow = False
        elif flagPaintRed is True:
            grabPaintX = x
            grabPaintY = y
            flagPaintRed = False
        else:    
            grabCordRDX = x
            grabCordRDY = y
            flagClickGrab = False

In [13]:
def procGrabUI():
    global flagGrab, flagDispGrab, flagClickGrab, flagPaintYellow, flagPaintRed, flagRect
    global grabCordLTX, grabCordLTY, grabCordRDX, grabCordRDY
    global grabPaintX, grabPaintY
    global grabImg, grabImg2, grabImg3
    global mask, bgdModel, fgdModel, rect
    
    if flagGrab is True:
        if EnableColorStream is True:
            grabImg = color_image.copy()
            grabImg3 = color_image.copy()
        elif EnableInfraredStream is True:
            grabImg = infrared_image.copy()
            grabImg3 = infrared_image.copy()
        
        cv2.namedWindow('GrabCutUI')
        cv2.setMouseCallback('GrabCutUI', procMouseGrab)
        cv2.imshow('GrabCutUI', grabImg)
        mask = np.zeros(grabImg.shape[:2], np.uint8)
        flagDispGrab= True
        flagGrab = False
    
    if flagDispGrab is True:
        
        if flagPaintYellow is True and flagRect is True:
            cv2.circle(grabImg, (grabPaintX,grabPaintY), thickness, (0, 255, 255), -1)
            cv2.circle(mask, (grabPaintX,grabPaintY), thickness, 1, -1)      
            cv2.imshow('GrabCutUI', grabImg)
        
        if flagPaintRed is True and flagRect is True:
            cv2.circle(grabImg, (grabPaintX,grabPaintY), thickness, (0, 0, 255), -1)
            cv2.circle(mask, (grabPaintX,grabPaintY), thickness, 0, -1)
            cv2.imshow('GrabCutUI', grabImg)
            mask[grabPaintY, grabPaintX] = 0

        if flagClickGrab is True:
            grabImg2 = grabImg.copy()
            if grabCordLTX != -1 and grabCordRDX != -1:
                cv2.rectangle(grabImg2, (grabCordLTX,grabCordLTY), (grabCordRDX, grabCordRDY), colorLineP, sizeLineP)
            cv2.imshow('GrabCutUI', grabImg2)
        else:
            if grabCordLTX != -1 and grabCordRDX != -1:
                cv2.rectangle(grabImg, (grabCordLTX,grabCordLTY), (grabCordRDX, grabCordRDY), colorLine, sizeLine)
                
                rect = (grabCordLTX, grabCordLTY, grabCordRDX, grabCordRDY)
                cv2.grabCut(grabImg3, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)            
                flagRect = True

                grabCordLTX = -1
                grabCordRDX = -1

            cv2.imshow('GrabCutUI', grabImg)

In [14]:
def procGrabCut():
    global grabImg3
    global mask, bgdModel, fgdModel, rect
    
    cv2.grabCut(grabImg3, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)   
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    grabImg3 = grabImg3 * mask2[:,:,np.newaxis]
    cv2.imshow('GRABCUT', grabImg3)

# utility function

In [15]:
# utility global parameters
utilityCoordX = -1
utilityCoordY = -1

In [16]:
# proc for development
def procDepthValue(event,x,y,flags,param):
    global utilityCoordX, utilityCoordY
    
    if event is cv2.EVENT_MOUSEMOVE:
        if x != 0 and y != 0:
            utilityCoordX = x
            utilityCoordY = y

In [17]:
if EnableColorStream is True:
    cv2.namedWindow('Color image')
    cv2.setMouseCallback('Color image', procDepthValue)

if EnableInfraredStream is True:
    cv2.namedWindow('Infrared image')
    cv2.setMouseCallback('Infrared image', procDepthValue) 
    
if EnableDepthStream is True:
    cv2.namedWindow('Depth image')
    cv2.setMouseCallback('Depth image', procDepthValue)

# main loop

In [18]:
try:
    while True:
        tick1 = cv2.getTickCount()
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        # Aligh frames
        aligned_frames = align.proccess(frames)

        if EnableColorStream is True:
            color_frame = aligned_frames.get_color_frame()
            if not color_frame:
                continue

        if EnableInfraredStream is True:
            infrared_frame = aligned_frames.first(rs.stream.infrared)
            if not infrared_frame:
                continue

        if EnableDepthStream is True:
            depth_frame = aligned_frames.get_depth_frame()
            if not depth_frame:
                continue

        # Convert images to numpy arrays
        if EnableColorStream is True:
            color_image = np.asanyarray(color_frame.get_data())

        if EnableInfraredStream is True:
            infrared_image = np.asanyarray(infrared_frame.get_data())

        if EnableDepthStream is True:
            depth_image = np.asanyarray(depth_frame.get_data())

        # colorize depth image
        depth_colormap = procColorMap(depth_image)

        # Screen capture
        procScreenCapture()

        # Painit function
        procMeasure()

        # Grabcut function
        procGrabUI()

        # FPS calculate
        tick2 = cv2.getTickCount()
        tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

        # Show images
        if EnableColorStream is True:
            temp = color_image.copy()

            if utilityCoordX <= DepthStreamResWidth and utilityCoordY <= DepthStreamResHeight:
                #cv2.circle(temp, (utilityCoordX,utilityCoordY), 5, colorLineP, -1)
                textString = str(depth_image[utilityCoordY, utilityCoordX])
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY), font, 1, colorDist, 1, cv2.LINE_AA)

            temp = cv2.copyMakeBorder(temp, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            textString = str(tick) + " ms"
            cv2.putText(temp, textString, (10, InfraredStreamResHeight+30), font, 1, colorFPS, 1, cv2.LINE_AA)

            cv2.imshow('Color image', temp)

        if EnableInfraredStream is True:
            temp = infrared_image.copy()

            if utilityCoordX <= DepthStreamResWidth and utilityCoordY <= DepthStreamResHeight:
                #cv2.circle(temp, (utilityCoordX,utilityCoordY), 5, colorLineP, -1)
                textString = str(depth_image[utilityCoordY, utilityCoordX])
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY), font, 1, colorDist, 1, cv2.LINE_AA)

            temp = cv2.copyMakeBorder(temp, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            textString = str(tick) + " ms"
            cv2.putText(temp, textString, (10, InfraredStreamResHeight+30), font, 1, colorFPS, 1, cv2.LINE_AA)

            cv2.imshow('Infrared image', temp)

        if EnableDepthStream is True:
            temp = depth_colormap.copy()

            if utilityCoordX <= DepthStreamResWidth and utilityCoordY <= DepthStreamResHeight:
                #cv2.circle(temp, (utilityCoordX,utilityCoordY), 5, colorLineP, -1)
                textString = str(depth_image[utilityCoordY, utilityCoordX])
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY), font, 1, colorDist, 1, cv2.LINE_AA)

            cv2.imshow('Depth image', temp)

        # Read device temperature
        '''
        temp = depth_sensor.get_option(rs.option.projector_temperature)
        print("proj: " + str(temp))
        temp = depth_sensor.get_option(rs.option.asic_temperature)
        print("asic: " + str(temp))
        '''

        # Generate point cloud
        #procPointCloud()


        # Keyboard command
        getKey = cv2.waitKey(10) & 0xFF
        if getKey is ord('c') or getKey is ord('C'):
            flagCapture = True
        elif getKey is ord('q') or getKey is ord('Q'):
            break
        elif getKey is ord('a') or getKey is ord('A'):
            flagMeasure = True
        elif getKey is ord('g') or getKey is ord('G'):
            flagGrab = True
        elif getKey is ord('h') or getKey is ord('H'):
            if flagRect is True:
                procGrabCut()
        elif getKey is ord('x') or getKey is ord('X'):
            cv2.destroyAllWindows()
            flagDispMeasure = False
            flagDispGrab = False

            if EnableColorStream is True:
                cv2.namedWindow('Color image')
                cv2.setMouseCallback('Color image', procDepthValue)

            if EnableInfraredStream is True:
                cv2.namedWindow('Infrared image')
                cv2.setMouseCallback('Infrared image', procDepthValue) 

            if EnableDepthStream is True:
                cv2.namedWindow('Depth image')
                cv2.setMouseCallback('Depth image', procDepthValue)
                
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipeline.stop()